In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/KT_aivle/딥러닝/RNN/미니프로젝트/train.csv')
df.head()

,text,label
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",코드2
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,웹
2,glob.glob(PATH) 를 사용할 때 질문입니다.\n\nPATH에 [ ] 가 ...,코드2
3,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals...",코드2
4,filename = TEST_IMAGE + str(round(frame_sec)) ...,코드2


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 22.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification


# 레이블 딕셔너리 정의
label_dict = {
        "코드1" : 0,
        "코드2" : 0,
        "웹" : 1,
        "이론" : 2,
        "시스템 운영" : 3,
        "원격" : 4
    }

# 레이블을 숫자로 변환
df["label"] = df["label"].map(label_dict)

# NLTK를 사용하여 불용어 제거
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english') # 불용어 리스트 가져오기

df["text"] = df["text"].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stopwords]))

# BERT 토크나이저 사용
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 입력 데이터 인코딩 및 패딩
X = df['text'].values
X_encodings = tokenizer.batch_encode_plus(X, padding=True, truncation=True, return_tensors='tf')

input_ids = X_encodings['input_ids']
attention_mask = X_encodings['attention_mask']

# 레이블 데이터 정의
y = df['label'].values

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# 학습/검증 데이터 분리
X_train_ids, X_val_ids, X_train_mask, X_val_mask, y_train, y_val = train_test_split(input_ids, attention_mask, y, test_size=0.25, random_state=42)

In [ ]:
X_train = {'input_ids': X_train_ids, 'attention_mask': X_train_mask}
X_val = {'input_ids': X_val_ids, 'attention_mask': X_val_mask}

In [ ]:
X_train = np.array(X_train)
X_val = np.array(X_val)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=1)

rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_val)

print(classification_report(y_val, y_pred))

TypeError: ignored

In [ ]:
len(X_encodings[0])

KeyError: ignored

In [ ]:

# BERT 모델 로드
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_dict))

# 모델 학습
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
history = model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), epochs=2, batch_size=32)

# 예측
y_pred = model.predict(X_val)[0].argmax(axis=-1)

# 모델 평가
print("Accuracy: {:.2f}%".format(accuracy_score(y_val, y_pred) * 100))
print("Precision: {:.2f}%".format(precision_score(y_val, y_pred, average='weighted') * 100))
print("Recall: {:.2f}%".format(recall_score(y_val, y_pred, average='weighted') * 100))
print("F1: {:.2f}%".format(f1_score(y_val, y_pred, average='weighted') * 100))

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2


ResourceExhaustedError: ignored